In [1]:
from sodapy import Socrata
import requests, zipfile, StringIO
import urllib2
import pandas as pd
import numpy as np
import pypyodbc
import csv
import os
import tempfile
import statsmodels.formula.api as smf
import pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
puma_ref = pd.read_excel('http://www.nyc.gov/html/dcp/download/census/nyc2010census_tabulation_equiv.xlsx',header=3)
# alternatively, use shapefile from https://data.cityofnewyork.us/Housing-Development/Public-Use-Microdata-Areas-PUMA-/cwiz-gcty
puma_list = puma_ref['PUMA'].unique()[~np.isnan(puma_ref['PUMA'].unique())]

In [3]:
pums_data = pd.read_csv('ss14hny.csv')
pums_data['PUMA'] = pums_data[['PUMA00','PUMA10']].max(axis=1)

In [4]:
nyc_pums = pd.DataFrame()
for puma in puma_list:
    nyc_pums = nyc_pums.append(pums_data.query('PUMA == @puma'))
    print nyc_pums.shape[0],'rows after addition of PUMA',str(puma)

2532 rows after addition of PUMA 3704.0
4290 rows after addition of PUMA 3706.0
6056 rows after addition of PUMA 3705.0
7926 rows after addition of PUMA 3703.0
9866 rows after addition of PUMA 3708.0
12388 rows after addition of PUMA 3702.0
14586 rows after addition of PUMA 3707.0
17711 rows after addition of PUMA 3710.0
20042 rows after addition of PUMA 3701.0
22819 rows after addition of PUMA 3709.0
26492 rows after addition of PUMA 4017.0
30173 rows after addition of PUMA 4013.0
32776 rows after addition of PUMA 4003.0
35635 rows after addition of PUMA 4014.0
37778 rows after addition of PUMA 4018.0
41340 rows after addition of PUMA 4004.0
43637 rows after addition of PUMA 4007.0
45814 rows after addition of PUMA 4002.0
50028 rows after addition of PUMA 4009.0
52591 rows after addition of PUMA 4005.0
55175 rows after addition of PUMA 4006.0
57103 rows after addition of PUMA 4011.0
59846 rows after addition of PUMA 4008.0
62467 rows after addition of PUMA 4010.0
65477 rows after addi

In [5]:
AMI_60_max = {1:36300,2:41460,3:46620,4:51780,5:56940,6:62100,7:67260,8:72420}
# 80 pct figures for 2015 from http://www1.nyc.gov/assets/hpd/downloads/pdf/NSP-tenant-lease-guidelines.pdf
AMI_80_max = {1:48400,2:55280,3:62160,4:69040,5:74640,6:80160,7:85680,8:91200}
nyc_pums['AMI_60'] = 0
for n in AMI_60_max.items():
    nyc_pums.loc[(nyc_pums['NP']==n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_60'] = 1
nyc_pums.loc[(nyc_pums['NP']>n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_60'] = 1
nyc_pums['AMI_80'] = 0
for n in AMI_80_max.items():
    nyc_pums.loc[(nyc_pums['NP']==n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_80'] = 1
nyc_pums.loc[(nyc_pums['NP']>n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_80'] = 1

In [28]:
age_grouping = {1:'PreWar',2:'PostWar',3:'PostWar',4:'PostWar',5:'PostWar',6:'PostWar',7:'PostWar',8:'Modern'}
nyc_pums['AgeGroup'] = ''
for n in age_grouping.items():
    nyc_pums.loc[nyc_pums['YBL']==n[0],'AgeGroup'] = n[1]
nyc_pums.loc[nyc_pums['YBL']>n[0],'AgeGroup'] = n[1]

In [21]:
size_grouping = {1:'Small',2:'Small',3:'Small',4:'Small',5:'Medium',6:'Medium',7:'Medium',8:'Large',9:'Large'}
nyc_pums['SizeGroup'] = ''
for n in size_grouping.items():
    nyc_pums.loc[nyc_pums['BLD']==n[0],'SizeGroup'] = n[1]
nyc_pums.loc[nyc_pums['BLD']>n[0],'SizeGroup'] = n[1]

In [8]:
nyc_pums['ResidencyGroup'] = ''
nyc_pums.loc[nyc_pums['MV']<=4,'ResidencyGroup'] = 'Under10'
nyc_pums.loc[nyc_pums['MV']==5,'ResidencyGroup'] = '10Years'
nyc_pums.loc[nyc_pums['MV']==6,'ResidencyGroup'] = '20Years'
nyc_pums.loc[nyc_pums['MV']==7,'ResidencyGroup'] = '30Years'

In [9]:
print ('Median HH income in all NYC PUMAs')
nyc_pums['HINCP'].median()

Median HH income in all NYC PUMAs


52000.0

In [10]:
print('Example median HH income in Williamsburg/Greenpoint')
nyc_pums[nyc_pums['PUMA10']==4001]['HINCP'].median()

Example median HH income in Williamsburg/Greenpoint


60000.0

In [15]:
# excluding group quarters and vacant household records
nyc_pums = nyc_pums.query('TYPE == 1 & BLD > 1 & BLD < 10')
nyc_pums = nyc_pums[nyc_pums['VACS'].isnull()]

In [29]:
print 'Percentage of households who own that are considered Low Income:', nyc_pums.query('TEN < 3')['AMI_80'].mean()
nyc_pums.query('TEN < 3').pivot_table(index='SizeGroup',columns='AgeGroup',values='AMI_80',aggfunc='mean')

Percentage of households who own that are considered Low Income: 0.366788757067


AgeGroup,Modern,PostWar,PreWar
SizeGroup,,,
Large,0.486380,0.675143,0.675643
Medium,0.617682,0.658810,0.610899
Small,0.669666,0.639246,0.643574


In [30]:
print 'Percentage of households who rent that are considered Low Income:', nyc_pums.query('TEN == 3')['AMI_80'].mean()
nyc_pums.query('TEN == 3').pivot_table(index='SizeGroup',columns='AgeGroup',values='AMI_80',aggfunc='mean')

Percentage of households who rent that are considered Low Income: 0.649729247627


AgeGroup,Modern,PostWar,PreWar
SizeGroup,,,
Large,0.486380,0.675143,0.675643
Medium,0.617682,0.658810,0.610899
Small,0.669666,0.639246,0.643574


In [14]:
# can we identify renters who live in condo buildings?
nyc_pums.query('TEN == 3 and FCONP == 1')
# it appears not

,insp,RT,SERIALNO,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP77,WGTP78,WGTP79,WGTP80,PUMA,AMI_60,AMI_80,AgeGroup,SizeGroup,ResidencyGroup


In [11]:
# dependents to test: HINCP, RNTP, GRNTP, NPF, TAXP
# independents to test: YBL (ord), TEN (c), BDSP, BLD, VACS - perhaps as a filter?
# also of interest: MV

In [12]:
# predict rent paid for renters
mod = smf.ols(formula='RNTP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                   RNTP   R-squared:                       0.282
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     430.1
Date:                Sat, 27 Feb 2016   Prob (F-statistic):               0.00
Time:                        01:29:21   Log-Likelihood:            -6.1558e+05
No. Observations:               79022   AIC:                         1.231e+06
Df Residuals:                   78949   BIC:                         1.232e+06
Df Model:                          72                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept         481.2329    413.640     

In [13]:
# predict income for renters
mod = smf.ols(formula='HINCP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  HINCP   R-squared:                       0.163
Model:                            OLS   Adj. R-squared:                  0.163
Method:                 Least Squares   F-statistic:                     214.1
Date:                Sat, 27 Feb 2016   Prob (F-statistic):               0.00
Time:                        01:29:28   Log-Likelihood:            -9.9640e+05
No. Observations:               79022   AIC:                         1.993e+06
Df Residuals:                   78949   BIC:                         1.994e+06
Df Model:                          72                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept        3.996e+04   5.12e+04     

In [14]:
# predict income for owners
mod = smf.ols(formula='HINCP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']<3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  HINCP   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     160.2
Date:                Sat, 27 Feb 2016   Prob (F-statistic):               0.00
Time:                        01:29:46   Log-Likelihood:            -6.5772e+05
No. Observations:               50376   AIC:                         1.316e+06
Df Residuals:                   50304   BIC:                         1.316e+06
Df Model:                          71                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept        2.567e+04   8.02e+04     

In [31]:
# predict income for owners with property tax as independent, but without bedrooms or size of building (covairance)
mod = smf.ols(formula='HINCP ~ TAXP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']<3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  HINCP   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.166
Method:                 Least Squares   F-statistic:                     143.7
Date:                Sun, 28 Feb 2016   Prob (F-statistic):               0.00
Time:                        11:02:03   Log-Likelihood:            -6.5629e+05
No. Observations:               50236   AIC:                         1.313e+06
Df Residuals:                   50165   BIC:                         1.313e+06
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept        8.861e+04   8.08e+04     

In [32]:
# predict rent paid for renters with grouped building ages
mod = smf.ols(formula='RNTP ~ BLD + BDSP + C(PUMA) + C(AgeGroup)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                   RNTP   R-squared:                       0.283
Model:                            OLS   Adj. R-squared:                  0.283
Method:                 Least Squares   F-statistic:                     534.6
Date:                Sun, 28 Feb 2016   Prob (F-statistic):               0.00
Time:                        11:02:19   Log-Likelihood:            -6.1148e+05
No. Observations:               78485   AIC:                         1.223e+06
Df Residuals:                   78426   BIC:                         1.224e+06
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                876

In [34]:
# predict rent paid for renters with grouped building ages and incumbency
mod = smf.ols(formula='RNTP ~ BDSP + C(SizeGroup) + C(AgeGroup) + C(PUMA) + C(MV)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                   RNTP   R-squared:                       0.377
Model:                            OLS   Adj. R-squared:                  0.376
Method:                 Least Squares   F-statistic:                     729.0
Date:                Sun, 28 Feb 2016   Prob (F-statistic):               0.00
Time:                        11:03:16   Log-Likelihood:            -6.0600e+05
No. Observations:               78485   AIC:                         1.212e+06
Df Residuals:                   78419   BIC:                         1.213e+06
Df Model:                          65                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                682

In [39]:
logit_mod = smf.logit(formula='AMI_80 ~ C(SizeGroup) + C(AgeGroup) + C(PUMA)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(logit_mod.summary())

         Current function value: 0.589822
         Iterations: 35


LinAlgError: Singular matrix

In [93]:
mfx = logit_mod.get_margeff()
print mfx.summary()

        Logit Marginal Effects       
Dep. Variable:                 AMI_80
Method:                          dydx
At:                           overall
                            dy/dx    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
C(SizeGroup)[T.Medium]    -0.0193      0.004     -4.912      0.000        -0.027    -0.012
C(SizeGroup)[T.Small]     -0.0531      0.005    -10.629      0.000        -0.063    -0.043
C(AgeGroup)[T.Modern]     -0.0481      0.007     -6.726      0.000        -0.062    -0.034
C(AgeGroup)[T.PostWar]    -0.0139      0.005     -2.839      0.005        -0.024    -0.004
C(AgeGroup)[T.PreWar]     -0.0172      0.005     -3.519      0.000        -0.027    -0.008
C(PUMA)[T.2602]           -1.6972   7.88e+04  -2.15e-05      1.000     -1.54e+05  1.54e+05
C(PUMA)[T.3701]           -5.7872   7.84e+04  -7.38e-05      1.000     -1.54e+05  1.54e+05
C(PUMA)[T.3702]           -5.